In [1]:
import sys
import os
import time
import numpy as np
import pandas as pd

import torchio as tio

import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms, utils
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, BatchSampler, Sampler

sys.path.insert(0, '../../models/')
sys.path.insert(0, '../../scripts/')
sys.path.insert(0, '../../')

from helpers import miscellaneous as misc
from helpers import plotters
from helpers import preprocessing
CONFIG = misc.get_config()

from ModelUtils import EncoderCNN
from ModelUtils import CognitiveTestMLP
from ModelUtils import FusionModel

#get Loader.py
from Loader import MRIDataset

#get TrainUtils.py helpers
from TrainUtils import train

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
print(device)

idx_to_label = {
    'CN': 0,
    'MCI': 1,
    'AD': 2
}

cuda


In [3]:
#transforms = None # equals no image pre-processing
preprocessor = preprocessing.ADNIPreprocessing(CONFIG)
#transforms = transforms.Compose([preprocessor.normalise()])

# Initialize Image Transformation for model
transforms = (
    tio.CropOrPad((50, 50, 50)),
    tio.RescaleIntensity(out_min_max=(-1, 1), in_min_max=(-1000, 1000))
)
transforms = tio.Compose(transforms)

In [4]:
# Set paths to results
# Add this to config file
metrics_path = CONFIG['METRICS_DIR']
saved_models_path = CONFIG['SERIALIZED_MODELS']

#get dataset 
#change parameters to match config file
dataset = MRIDataset("../../" + CONFIG['TRAIN_LABELS_DIR'], transform = transforms)

#return dataloader with given parameters
train_loader = DataLoader(
    dataset = dataset,
    batch_size = 64, 
    num_workers = 4, 
    shuffle = True,
    pin_memory = True,
)

learning_rate = 0.01
max_epochs = 1

def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear) or isinstance(m, nn.Conv3d):
        torch.nn.init.xavier_uniform_(m.weight.data)

# Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model = EncoderCNN(3).to(device)
model.apply(weights_init)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
print("length dataset: ", len(dataset))

length dataset:  1699


Process until it starts training is really long. So maybe another approach is better.

In [ ]:
# %%time

# Train model
results = train(train_loader, model, optimizer, max_epochs, device)

In [7]:
# Save model
run_name = "TEST-NeuronFusionExcessiveGigaNet-18"
results.to_csv("../../" + metrics_path + run_name + ".csv", index=False)
torch.save(model, "../../" + saved_models_path + run_name + ".pth")